In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/dacondata/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/dacondata/test.csv', index_col=0)
sample_submission = pd.read_csv('/kaggle/input/daconsub/sample_submission.csv', index_col=0)

In [ ]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [ ]:
train.head()

In [ ]:
train['type'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20,12))
sns.boxplot(train['type'],train['petroMag_g'])

#sns.distplot(train['petroMag_g'])

In [ ]:
train['petroMag'] = (train['petroMag_u']+train['petroMag_g']+train['petroMag_r']+train['petroMag_i']+train['petroMag_z'])/5
test['petroMag'] = (test['petroMag_u']+test['petroMag_g']+test['petroMag_r']+test['petroMag_i']+test['petroMag_z'])/5

# 평균을 구하면 petromag의 대푯값이 나옴. 변수를 잘 설명하는 칼럼을 만들면 학습에 더 도움 
# 비슷한 성격을 가진 애들끼리 모아 새로운 변수 만들어 
# tree 약점 독립적인 칼럼에 대해 학습은 잘하는 데 여러가지 칼럼을 동시에 고려 아니면 그 변수를 조합한 새로운 변수를 학습하는 건 잘 못함
# 집값 예측 시, 제 3의 정보를 만드느냐 못만드냐 차이 
# 교호작용 : 파생정보를 만들 수 있냐 없냐를 판단할 수 있는 

In [ ]:
train.head()

In [ ]:
train2 = train.drop(columns=['type', 'type_num', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train2, train['type_num'], stratify=train['type_num'], test_size = 0.2, random_state = 1)
# 20:18 학습 시, 정답의 분포가 중요함.
# 비율에 맞춰 정답을 뽑아야함 -> stratify = train['type_num'] 이용
# 

In [ ]:
y_valid.value_counts()

In [ ]:
x_train.shape

In [ ]:
train2.head()

In [ ]:
test2.head()

In [ ]:
train2.shape


In [ ]:
test2.shape

In [ ]:
test2 = test.drop(columns=['petroMag'], axis=1)

In [ ]:
test2.head()

In [ ]:
#from sklearn.preprocessing import LabelEncoder

In [ ]:
#le = LabelEncoder()
#le.fit(train['type'])
#train['type'] = le.transform(train['type'])

In [ ]:
train = train.drop(columns=['type', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z'], axis=1)
train.head()

In [ ]:
train2.head()

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#rf = RandomForestClassifier(n_estimators = 100, n_jobs = 4)
#rf.fit(x_train, y_train)

In [ ]:
# 20:24
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='gpu_hist', n_estimators= 10000) # gpu를 사용하는 #max_depth는 기본이 3으로 설정
xgb.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='mlogloss', early_stopping_rounds=10  ) 
# eval_metric 로그로스 / # 학습을 많이해줄수록 로스가 떨어지는 게 보임 -> 학습 더 많이 진행 / early_stopping_rounds - 가장 좋은 점수 그 이상으로 나오지 않으면 멈춤
# 평가셋에서는 0.37이 나왔지만 테스트에서는 더 안좋게 나옴 -> 그래서 정답을 비율에 맞춰서 뽑아 진행
# max_depth를 세분화하여 모델의 성능을 높임(학습을 복잡하게)


# 3일때 1059

In [ ]:
# # 20:39 # max_depth를 세분화하여 모델의 성능을 높임(학습을 복잡하게)
# from xgboost import XGBClassifier
# xgb = XGBClassifier(tree_method='gpu_hist', max_depth=6, n_estimators= 10000, colsample_bytree=0.8, subsample=0.7) # 나무를 만들때마다 칼럼을 샘플링 진행 /
# # tree는 greedy라 학습이 잘되는 x변수만 학습하게됨, 이에 train에서는 동작잘할 수 있어도 test에서는 그게 아닐 수 있음. 이걸 방지하는 게 colsample_bytree = 칼럼을 랜덤하게 80%를 뽑겠다
# # 이전에는 중요한 칼럼만 학습하는 경우가 있었는데(값이 튀는 칼럼이 나왔을때, 또는 피쳐들에 대비x) 이제는 80%비율로 모든 칼럼을 학습함. 이를 비롯 일반화 능력이 상승 
# # 트레인셋과 테스트 셋이 비슷할 때(=분포 비슷) 랜덤하게 뽑는게 도움이 안될 수 잇음
# # colsample_bytree 이건 최적의 값이라는 게 없고 적당히 어떤 값만 넣어주면 됨 
# # colsample_bytree 혜택 -> 하나의 나무를 만들때 다른 나무와 다르게 다양하게 학습(=앙상블 효과, 다양하게 질문해 학습)
# # subsample 이건 데이터도 랜덤하게 뽑아 최대한 다양한 데이터를 학습하게, 이것도 최적 값이라는 게 없음 
# xgb.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='mlogloss', early_stopping_rounds=10  ) 

# max_depth 6일땐 334개


In [ ]:
# 20:51 # max_depth = 9 
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='gpu_hist', max_depth=9, n_estimators= 10000, colsample_bytree=0.4, subsample=0.6, learning_rate=0.01) # 뎁스가 9면 나무의 가지가 많이 필요치는 않음 
# depth를 나눠준 이유는 학습 방식을 다양하게(단순, 중간복잡, 복잡)
# 이에 colsample_bytree, 등 기타 옵션들도 바꿔줘야함 
# 새로운 옵션(***하이퍼파라미터, learning rate가 중요함, #스키를 타고 내려오면서, 이 내려오는 속도를 결정 - 
# 속도 빠르면 빠르게 최적이라 말할 수 있는 곳으로 수렴, 아니면 빠르게 점핑 점핑(스탭)해서 발산 또는 그 주변만 왔다갔다 
# 속도 느리면 최적지점 도달까지 오랜 시간 걸리지만, 최적의 지점까지 잘 갈 수 있다. 근데 local 미니멈에 멈출 수 있음 ?국소지역해?
# 보통 0.01 ~ 0.25가 좋음
# learning rate scheduler로 학습속도 조절 가능
xgb.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='mlogloss', early_stopping_rounds=10  ) 

# depth 9일때, 나무 갯수 최적 : 1929개 

In [ ]:
# 앙상블 효과 - > 합치면 좋은 결과가 나옴 (잘 못한 학습 (max =9 일때)을 다른 데에선 잘 학습할 수 있게 도와줌)


# max_depth 9일떄 / 앙상블 했을 떄


In [ ]:
# 21:18 # max_depth = 9 
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='gpu_hist', max_depth=9, n_estimators= 1929, colsample_bytree=0.4, subsample=0.6, learning_rate=0.01) # 뎁스가 9면 나무의 가지가 많이 필요치는 않음 
# depth를 나눠준 이유는 학습 방식을 다양하게(단순, 중간복잡, 복잡)
# 이에 colsample_bytree, 등 기타 옵션들도 바꿔줘야함 
# 새로운 옵션(***하이퍼파라미터, learning rate가 중요함, #스키를 타고 내려오면서, 이 내려오는 속도를 결정 - 
# 속도 빠르면 빠르게 최적이라 말할 수 있는 곳으로 수렴, 아니면 빠르게 점핑 점핑(스탭)해서 발산 또는 그 주변만 왔다갔다 
# 속도 느리면 최적지점 도달까지 오랜 시간 걸리지만, 최적의 지점까지 잘 갈 수 있다. 근데 local 미니멈에 멈출 수 있음 ?국소지역해?
# 보통 0.01 ~ 0.25가 좋음
# learning rate scheduler로 학습속도 조절 가능
xgb.fit(train2, train['type_num']) 

# depth 9일때, 나무 갯수 최적 : 1929개 

In [ ]:
result = xgb.predict_proba(test2)

In [ ]:
# 21:32 # max_depth를 세분화하여 모델의 성능을 높임(학습을 복잡하게)
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='gpu_hist', max_depth=6, n_estimators= 334*5, learning_rate=0.02, colsample_bytree=0.8, subsample=0.7) # 나무를 만들때마다 칼럼을 샘플링 진행 /
# tree는 greedy라 학습이 잘되는 x변수만 학습하게됨, 이에 train에서는 동작잘할 수 있어도 test에서는 그게 아닐 수 있음. 이걸 방지하는 게 colsample_bytree = 칼럼을 랜덤하게 80%를 뽑겠다
# 이전에는 중요한 칼럼만 학습하는 경우가 있었는데(값이 튀는 칼럼이 나왔을때, 또는 피쳐들에 대비x) 이제는 80%비율로 모든 칼럼을 학습함. 이를 비롯 일반화 능력이 상승 
# 트레인셋과 테스트 셋이 비슷할 때(=분포 비슷) 랜덤하게 뽑는게 도움이 안될 수 잇음
# colsample_bytree 이건 최적의 값이라는 게 없고 적당히 어떤 값만 넣어주면 됨 
# colsample_bytree 혜택 -> 하나의 나무를 만들때 다른 나무와 다르게 다양하게 학습(=앙상블 효과, 다양하게 질문해 학습)
# subsample 이건 데이터도 랜덤하게 뽑아 최대한 다양한 데이터를 학습하게, 이것도 최적 값이라는 게 없음 
xgb.fit(train2, train['type_num']) 

# max_depth 6일땐 334개

# result2로 저장해서 result와 앙상블 

In [ ]:
result2 = xgb.predict_proba(test2)

In [ ]:
# %%time

# from xgboost import XGBClassifier
# rf = XGBClassifier(tree_method = 'gpu_hist', max_depth = 10, n_estimators = 200)
# rf.fit(train2, train['type_num'])

In [ ]:
#from sklearn.metrics import log_loss
#log_loss(pd.get_dummies(pd.DataFrame(y_valid).astype('object')), result)

In [ ]:
#sub = pd.read_csv('/kaggle/input/sample_submission.csv')
#sub.iloc[: , 1:] = result
#sub.head()

In [ ]:
submission = pd.DataFrame(data=result*0.9+result2*0.1, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('sub_ens.csv', index=True)

In [ ]:
#pd.Series(rf.feature_importances_, index = train2.columns).sort_values(ascending = False)

In [ ]:
# lgbm + xg 합쳐서
# xg learning = 0.01로 맥스 뎁스 최적 값 찾구 

# 딥러닝 -> 

In [0]:
import pandas as pd
sample_submission = pd.read_csv("../input/sample_submission.csv")